In this notebook, I classified all transcripts using zero-shot classification. For this step, I did not fine-tuned the model, but just used the pre-trained zero-shot classification to see how the model itself can classify each transcripts.

In [ ]:
import pandas as pd

In [ ]:
! pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 35.7 MB/s 
     |████████████████████████████████| 6.6 MB 42.6 MB/s 
     |████████████████████████████████| 895 kB 14.9 MB/s 
     |████████████████████████████████| 77 kB 4.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
basic_df = pd.read_csv('/content/drive/MyDrive/Capstone Design/data/processed_data/processed_data.csv')
chunk_df = pd.read_csv('/content/drive/MyDrive/Capstone Design/data/processed_data/all_conv_df.csv')

In [ ]:
basic_df = basic_df.drop('Unnamed: 0', axis = 1)
basic_df = basic_df[~basic_df['speech'].isna()]

In [ ]:
chunk_df = chunk_df.drop('Unnamed: 0', axis = 1)

In [ ]:
basic_df.head()

,participant,file_name,speaker,speech,date,transcript_filepath,year,month,day
0,2039,2039_parent_11.04.2021.docx,Speaker 1:,"Has been involved in, I would love to hear abo...",2021-11-4,/content/drive/MyDrive/Capstone Design/data/fi...,2021,11,4
1,2039,2039_parent_11.04.2021.docx,Speaker 2:,"Correct. No, he's post high school.",2021-11-4,/content/drive/MyDrive/Capstone Design/data/fi...,2021,11,4
2,2039,2039_parent_11.04.2021.docx,Speaker 1:,Just like to confirm. So did Adrian exit the s...,2021-11-4,/content/drive/MyDrive/Capstone Design/data/fi...,2021,11,4
3,2039,2039_parent_11.04.2021.docx,Speaker 2:,"No, he was already out.",2021-11-4,/content/drive/MyDrive/Capstone Design/data/fi...,2021,11,4
4,2039,2039_parent_11.04.2021.docx,Speaker 1:,"Okay. So for this section, I'm going to ask yo...",2021-11-4,/content/drive/MyDrive/Capstone Design/data/fi...,2021,11,4


In [ ]:
chunk_df.head()

,filename,passage
0,2039_parent_11.04.2021.docx,"Has been involved in, I would love to hear abo..."
1,2033_VocIndex-and-SA_07.09.2020.docx,"... wash, that's probably one of the more... H..."
2,2035_GAS-and-VocIndex_06.18.2020.docx,"No problem. Great. Hey, Molly. That was quick...."
3,2040_VocIndex_06.19.2020.docx,"Okay. I'm guessing not as well, because usuall..."
4,2034_GAS_Voc_SA_05.18.2020.docx,That maybe you're not getting now. So hold on ...


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
chunk_df['passage'][0]

'Has been involved in, I would love to hear about what he\'s doing now. So as of the last interview,  was not in high school. That is still correct? Correct. No, he\'s post high school. Just like to confirm. So did Adrian exit the school system since we last spoke in May? No, he was already out. Okay. So for this section, I\'m going to ask you about academic programs, employment, and volunteering activities that Adrian attended since May 12th of this year in 2021. So first, I\'d like to get an idea of how Adrian usually spends his time. We\'re going to talk about the activities that he participates in during a typical week. So what does a typical week look like for Adrian? How has he been spending his time? Well, I\'m going guess on some of this, because I\'m at work during the day. So I\'m not sure what he does when I\'m at work. But from my understanding, what he does when I\'m at work is, he\'ll work out. I don\'t think it\'s as consistent as he says. No, I shouldn\'t say that. He i

In [ ]:
sequence_to_classify = chunk_df['passage'][0]
candidate_labels = ['self-determinant', 'dependent', 'neutral']
test = classifier(sequence_to_classify, candidate_labels)

In [ ]:
test['scores'][0]

0.5063992142677307

In [ ]:
self_list = []
depend = []
neutral = []

for conv in range(chunk_df.shape[0]):
  sequence_to_classify = chunk_df['passage'][conv]
  candidate_labels = ['self-determinant', 'dependent', 'neutral']
  score = classifier(sequence_to_classify, candidate_labels)
  self_list.append(score['scores'][0])
  depend.append(score['scores'][1])
  neutral.append(score['scores'][2])

In [ ]:
chunk_df['self-determination'] = self_list
chunk_df['dependent'] = depend
chunk_df['neutral'] = neutral

In [ ]:
def categories(row):
  s = row['self-determination'] 
  d = row['dependent']
  n = row['neutral']
  if max(s, d, n) == s:
    return 'self-determination'
  elif max(s, d, n) == d:
    return 'dependent'
  else:
    return 'neutral'

In [ ]:
def category_score(row):
  s = row['self-determination'] 
  d = row['dependent']
  n = row['neutral']
  if max(s, d, n) == s:
    return s
  elif max(s, d, n) == d:
    return d
  else:
    return n

In [ ]:
chunk_df['category'] = chunk_df.apply(lambda row: categories(row), axis = 1)

In [ ]:
chunk_df['score'] = chunk_df.apply(lambda row: category_score(row), axis = 1)

In [ ]:
chunk_df.head()

,filename,passage,self-determination,dependent,neutral,category,score
0,2039_parent_11.04.2021.docx,"Has been involved in, I would love to hear abo...",0.506399,0.339295,0.154306,self-determination,0.506399
1,2033_VocIndex-and-SA_07.09.2020.docx,"... wash, that's probably one of the more... H...",0.437495,0.284159,0.278345,self-determination,0.437495
2,2035_GAS-and-VocIndex_06.18.2020.docx,"No problem. Great. Hey, Molly. That was quick....",0.521456,0.346692,0.131852,self-determination,0.521456
3,2040_VocIndex_06.19.2020.docx,"Okay. I'm guessing not as well, because usuall...",0.447407,0.316077,0.236516,self-determination,0.447407
4,2034_GAS_Voc_SA_05.18.2020.docx,That maybe you're not getting now. So hold on ...,0.406271,0.320712,0.273018,self-determination,0.406271


In [ ]:
print(chunk_df.groupby('category').size())

category
self-determination    293
dtype: int64


In [ ]:
chunk_df = chunk_df.sort_values('score', ascending = False)

In [ ]:
chunk_df.reset_index(drop = True)

,filename,passage,self-determination,dependent,neutral,category,score
0,3007_Parent-Interview-T3_10.04.2021.docx,... involved. And I don't even have that much ...,0.948454,0.026206,0.025340,self-determination,0.948454
1,1057_voc index_11.22.2021.docx,So is he currently working? No. And this is ju...,0.932546,0.037624,0.029831,self-determination,0.932546
2,1069_voc index_11.29.2021.docx,Just wanting to confirm that she is still in h...,0.905839,0.051979,0.042183,self-determination,0.905839
3,1004_voc index_07.14.2021.docx,"And so just to confirm, he is still in high sc...",0.899324,0.050638,0.050039,self-determination,0.899324
4,1058_Voc_Service_08.04.2020.docx,Worked during high school? No. And what about ...,0.889577,0.059236,0.051187,self-determination,0.889577
...,...,...,...,...,...,...,...
288,1028_Service_VocIndex_02.14.2020.docx,To something different where we're going to ta...,0.351899,0.350452,0.297649,self-determination,0.351899
289,2056_VocIndex_06.23.2020.docx,History. So did he... Has Marshall worked duri...,0.348141,0.329169,0.322690,self-determination,0.348141
290,1041_voc index_05.11.2021.docx,Just confirming he is out of high school? Did ...,0.347981,0.326496,0.325523,self-determination,0.347981
291,3018_Check-In-1-Interview_08.16.2021.docx,Make sure I didn't miss anything. It'll tell m...,0.341792,0.338323,0.319885,self-determination,0.341792


In [ ]:
chunk_df.to_csv('zero-shot-one-conv.csv')

# Individual line

In [ ]:
import pandas as pd
chunk_df = pd.read_csv('/content/drive/MyDrive/Capstone Design/data/processed_data/all_conv_df.csv')

In [ ]:
def sen_max(row):
  test = row['passage'].split(". ")
  candidate_labels = ['self-determinant', 'dependent', 'neutral']

  for speech in range(len(test)):
    if len(test[speech]) > 5:
      max_index = -1
      max_score = -1
      score = classifier(test[speech], candidate_labels)
      s = score['scores'][0]
      d = score['scores'][1]
      n = score['scores'][2]
      if max(s, d, n) == s:
        if s > max_score:
          max_index = speech
          max_score = s
      print(str(speech) + ' Done')
  return [max_score, test[max_index]]

In [ ]:
test = chunk_df.sample(100)

In [ ]:
test

,Unnamed: 0,filename,passage
267,267,2022_SA-and-Voc_05.23.2020.docx,"Well, so first I'm going to ask you, has he ev..."
19,19,2023_voc_SA_GAS_02.17.2020.docx,"Okay, thanks. Participants 2023. It's February..."
194,194,1022_voc index_01.05.2021.docx,"Okay. So, her high school situation, we've tal..."
35,35,1066_voc_index_11.22.2021.docx,... and are activities that he might be involv...
134,134,2053_parent_11.01.2021.docx,"... . Okay. Hey, you guys, we're in a Zoom me..."
...,...,...,...
200,200,1015_voc index_youth_08.24.2021.docx,"Visit, we set three annual goals with you. And..."
92,92,1030_voc index_05.20.2021.docx,I just wanted to confirm that he is still in h...
40,40,1063_VocYouth_08.10.2020.docx,"First I want to talk to you about, so as I und..."
216,216,1009_voc index_02.25.2021.docx,He's not currently working. Yeah. What's the r...


In [ ]:
test['result'] = test.apply(lambda row: sen_max(row), axis = 1)

0 Done
2 Done
4 Done
7 Done
9 Done
10 Done
11 Done
12 Done
13 Done
14 Done
15 Done
17 Done
18 Done
19 Done
20 Done
22 Done
23 Done
24 Done
25 Done
26 Done
27 Done
28 Done
29 Done
30 Done
31 Done
32 Done
33 Done
34 Done
35 Done
36 Done
37 Done
39 Done
40 Done
41 Done
44 Done
45 Done
47 Done
49 Done
50 Done
51 Done
52 Done
53 Done
54 Done
55 Done
56 Done
57 Done
59 Done
64 Done
65 Done
66 Done
67 Done
68 Done
69 Done
71 Done
72 Done
75 Done
77 Done
80 Done
81 Done
83 Done
0 Done
1 Done
2 Done
4 Done
5 Done
6 Done
7 Done
8 Done
11 Done
12 Done
14 Done
16 Done
17 Done
18 Done
20 Done
22 Done
23 Done
24 Done
25 Done
27 Done
29 Done
30 Done
33 Done
35 Done
36 Done
37 Done
42 Done
43 Done
44 Done
45 Done
46 Done
47 Done
48 Done
49 Done
51 Done
53 Done
54 Done
55 Done
56 Done
57 Done
58 Done
59 Done
60 Done
61 Done
62 Done
64 Done
67 Done
70 Done
71 Done
72 Done
74 Done
77 Done
78 Done
79 Done
81 Done
82 Done
83 Done
84 Done
85 Done
86 Done
87 Done
89 Done
90 Done
91 Done
92 Done
94 Done
96 Do

In [ ]:
test.to_csv('/content/drive/MyDrive/Capstone Design/data/processed_data/sen_zero_shot.csv')